<a href="https://colab.research.google.com/github/mipypf/practical-mi-guide/blob/develop/chapter4/src/llm_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAGによるLLMへの知識付与

## Google colabを使用の場合、ランタイムのタブから「ランタイムのタイプを変更」→ハードウェアアクセラレータと進み、T4 GPUを選択

In [1]:
# 以下のコマンドでGPUが使用可能かを確認
! nvidia-smi

Fri Jan 24 01:42:31 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Google Colabの準備

In [2]:
# Google Colabを利用する場合はTrue、そうでない場合はFalseとする
colab = True

In [3]:
# Google Colabのファイルをクリックし、MIハンドブック_はじめに.pdfをドラッグ＆ドロップしてアップロード
if colab:
  INPUT_FILE_PATH = "./"
  OUTPUT_FILE_PATH = "./"
else:
  INPUT_FILE_PATH = "../input/"
  OUTPUT_FILE_PATH = "../output/"

## ライブラリをインポート

In [21]:
!pip install faiss-cpu #執筆当時のGoogle Colabではfaiss-gpuのpip installではERRORが発生する
!pip install pypdf==4.1.0
!pip install fugashi #"tohoku-nlp/bert-base-japanese-char"の駆動に必要
!pip install ipadic #"tohoku-nlp/bert-base-japanese-char"の駆動に必要
!pip install unidic_lite #"tohoku-nlp/bert-base-japanese-char"の駆動に必要

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 14.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for unidic_lite: filename=unidic_lite-1.0.8-py3-none-any.whl size=47658818 sha256=b373710591727ebaf61ab5deea20eba26eb6f2617e548822f455b0efdbe7a7d7
  Stored in directory: /root/.cache/pip/wheels/b7/fd/e9/ea4459b868e6d2902e8d80e82dbacb6203e05b3b3a58c64966
Successfully built unidic_lite


In [16]:
import warnings

warnings.filterwarnings("ignore")

import re

import faiss
import numpy as np
import torch
from IPython.display import Markdown
from pypdf import PdfReader
from sentence_transformers import SentenceTransformer
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

## データを読み込み文章を一定のまとまり（チャンク）に分割し、前処理を実施

In [17]:
# PDFを読み込む関数
def load_pdf(file_path):
    reader = PdfReader(file_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text


# テキストをチャンクに分割する関数
def split_text(text, chunk_size=300):
    chunks = [text[i : i + chunk_size] for i in range(0, len(text), chunk_size)]
    return chunks


# PDFから抽出したテキストから改行コードを削除する関数
def clean_text(text):
    return text.replace("\n", "").strip()

In [18]:
# PDFファイルをロードしてチャンクに分割
file_path = INPUT_FILE_PATH + "MIハンドブック_はじめに.pdf"
text = load_pdf(file_path)
cleaned_text = clean_text(text)
documents = split_text(cleaned_text)

In [19]:
documents

['はじめに   良い材料を 早く創りたいので 、マテリアルズ・インフォマティクス（ MI）を活用して”いる”。このような声 はもう珍しくありません 。 MIブームに端を発して 、大学・企業 ともにMIの活用検討が 始められて 既にある程度の期間が 経過しました 。すでにMIに取り組んでいる 大学・企業 からはMI活用の成果 が多く報告されており 、MIは期待に応えられるということが共通理解になりつつあります。 MIとは？有効なものなのか？が大勢を占めていた数年前とは隔世の感があります。  本書を手に取っていただいた方は、 材料開発者として、 あるいは材料開発を支援する データ分析者として「MIを',
 '活用して 材料開発を加速したい。 」という気持ち だと考えています 。   著者の高原と福岡は 多くの民間企業などに対して MI活用の支援をさせて頂いています。 学生時代に は二人とも 材料工学を専攻 し、それぞれ別 メーカーでMIを活用した材料開発業務を経て、 現在はコンサルティング ・データ分析・講演・教育・データ活用システムの提供 といった 様々な立場 ・内容で活動を行って います。  MIの普及に伴い、様々な書籍やセミナーが 発刊・実施されています 。しかしそれは、MIとは？という概論か、 整形されたデータに対して分析アルゴリズムを適用する ものが主と感じています。  材料開発の実務の',
 '中で MIを活用するには、さらに踏み込んだ情報が必要です。 例えば、データの整形方法。 MIに限らず機械学習 ・データ分析の入門書では分析可能 な形に整形されたデータから分析がスタートします。一方で、 MIが普及していない 材料開発の現場では、 データ分析フレンドリーな 形でデータが蓄積されていません。そのため、そ のデータを分析可能にする手順が分からずMIの活用を断念してしまうことがあります 。 次にデータや分析結果の 見方という観点。 本書執筆の 時点では 、AIにデータを丸ごと投げ込み、出てきた結果をそのまま実験すればより良い材料が出来るということは 、ほとんどの場合 ありません 。手持',
 'ちのデータのどのような点に着目し、 データ分析を適用するか。出てきた分析結果を どのように 材料開発の加速に役立て ていくか。こういった ノウハウが必要に なります。

## Transformer系列のモデルを用いたベクトル化を行う

In [22]:
# SentenceTransformerを使用して埋め込みを生成
embedding_model = SentenceTransformer("tohoku-nlp/bert-base-japanese-char")
embeddings = embedding_model.encode(documents, convert_to_tensor=False)

In [23]:
embeddings

array([[-0.00860148, -0.01544502,  0.05732767, ...,  0.10766527,
         0.02744266,  0.07329489],
       [-0.0287408 , -0.03830467,  0.0481893 , ...,  0.10121735,
         0.02751626,  0.04565814],
       [-0.00059898, -0.09614202,  0.06609967, ...,  0.07873918,
        -0.14247678,  0.16064683],
       ...,
       [ 0.05802046, -0.14048982,  0.08772261, ...,  0.11404864,
        -0.02176702,  0.12419245],
       [ 0.05007084, -0.04091921, -0.0248207 , ...,  0.01192917,
         0.01065823,  0.05920728],
       [ 0.027572  ,  0.03278046,  0.03972064, ...,  0.0483238 ,
         0.09655842, -0.30393997]], dtype=float32)

## 検索ができるようにベクトル化したテキストデータにインデックスを付与

In [24]:
# FAISSインデックスを作成
d = embeddings[0].shape[0]  # 埋め込みの次元
index = faiss.IndexFlatL2(d)
index.add(np.array(embeddings))

## LLMを読み込む

### HuggingFaceでの作業
 - https://huggingface.co/google/gemma-2-2b-jpn-it にアクセスしてログインし、ライセンス認証を行う。（Singn up未の場合は、登録から行う。）
 - HuggingFaceのトークンをhttps://huggingface.co/settings/tokens から取得
 - Google Colabのシークレット（鍵マークのアイコン）をクリックして開き、「+ 新しいシークレットを追加」からHF_TOKENという名前を付けて、値の欄に取得したトークンを入力
 - ノートブックからのアクセスを有効にする

In [27]:
from google.colab import userdata

from huggingface_hub import login
login(token=userdata.get('HF_TOKEN'))

In [28]:
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b-jpn-it")
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2-2b-jpn-it",
    device_map="auto",
    torch_dtype=torch.bfloat16,
)

tokenizer_config.json:   0%|          | 0.00/46.9k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/555 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

## テキスト生成のための設定

In [29]:
# テキスト生成パイプラインを設定
text_gen_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

Device set to use cuda:0


In [30]:
def ask_question(question):
    # 質問を埋め込みに変換
    question_embedding = embedding_model.encode([question], convert_to_tensor=False)

    # FAISSで最も近いチャンクを検索
    _, indices = index.search(np.array(question_embedding), k=1)
    retrieved_text = documents[indices[0][0]]

    # 質問と関連情報、回答の間に改行を追加
    input_text = f"質問: {question}\n\n関連情報: {retrieved_text}\n\n回答: "

    # パイプラインを使用して回答を生成
    outputs = text_gen_pipeline(input_text, max_new_tokens=256)

    # リストから文字列を取得
    assistant_response = outputs[0]["generated_text"].strip()

    return assistant_response

## RAGを使用せず知識付与を行わない状態でLLMに質問する

In [31]:
question = "MIとは何ですか？"

outputs = text_gen_pipeline(question, max_new_tokens=256)
response = outputs[0]["generated_text"].strip()
print(response)

The 'batch_size' attribute of HybridCache is deprecated and will be removed in v4.49. Use the more precisely named 'self.max_batch_size' attribute instead.


MIとは何ですか？

「**AI**」の略称で、人工知能の技術です。

**AIとは？**

* **人間のような思考・判断能力を持つコンピュータ**
* **学習・分析・予測・行動**を自動化できる能力を持つ

**AIの活用例**

* **自動運転車**
* **医療診断**
* **顧客対応**
* **商品推薦**
* **翻訳**
* **画像認識**


In [32]:
display(Markdown(response))

MIとは何ですか？

「**AI**」の略称で、人工知能の技術です。

**AIとは？**

* **人間のような思考・判断能力を持つコンピュータ**
* **学習・分析・予測・行動**を自動化できる能力を持つ

**AIの活用例**

* **自動運転車**
* **医療診断**
* **顧客対応**
* **商品推薦**
* **翻訳**
* **画像認識**

## RAGを使用して知識付与を行った状態でLLMに質問する

In [33]:
print(ask_question(question))

質問: MIとは何ですか？

関連情報: はじめに   良い材料を 早く創りたいので 、マテリアルズ・インフォマティクス（ MI）を活用して”いる”。このような声 はもう珍しくありません 。 MIブームに端を発して 、大学・企業 ともにMIの活用検討が 始められて 既にある程度の期間が 経過しました 。すでにMIに取り組んでいる 大学・企業 からはMI活用の成果 が多く報告されており 、MIは期待に応えられるということが共通理解になりつつあります。 MIとは？有効なものなのか？が大勢を占めていた数年前とは隔世の感があります。  本書を手に取っていただいた方は、 材料開発者として、 あるいは材料開発を支援する データ分析者として「MIを

回答: 

MI（Materials Informatics）は、**材料の特性や性能を理解し、設計・開発を効率化するための、材料科学とデータ分析の融合**です。

**具体的には、以下の要素が含まれます:**

* **材料のデータベース:**  膨大な材料データの蓄積と管理
* **データ分析:**  材料データから、材料の特性や性能を分析
* **AI・機械学習:**  材料の設計・開発を支援するAIや機械学習モデルの開発
* **シミュレーション:**  材料の性能を予測するためのシミュレーション

**MIのメリット:**

* **材料開発の効率化:**  従来の材料開発プロセスを大幅に短縮
* **新しい材料の発見:**  膨大なデータから、新しい材料の発見が可能
* **材料性能の向上:**  材料の性能をより正確に予測・設計
* **コスト削減:**  材料開発の費用を削減


In [34]:
display(Markdown(ask_question(question)))

質問: MIとは何ですか？

関連情報: はじめに   良い材料を 早く創りたいので 、マテリアルズ・インフォマティクス（ MI）を活用して”いる”。このような声 はもう珍しくありません 。 MIブームに端を発して 、大学・企業 ともにMIの活用検討が 始められて 既にある程度の期間が 経過しました 。すでにMIに取り組んでいる 大学・企業 からはMI活用の成果 が多く報告されており 、MIは期待に応えられるということが共通理解になりつつあります。 MIとは？有効なものなのか？が大勢を占めていた数年前とは隔世の感があります。  本書を手に取っていただいた方は、 材料開発者として、 あるいは材料開発を支援する データ分析者として「MIを

回答: 

MI（Materials Informatics）は、**材料の特性や性能を理解し、設計・開発を効率化するための、材料科学とデータ分析の融合**です。

**具体的には、以下の要素が含まれます:**

* **材料のデータベース:**  膨大な材料データの蓄積と管理
* **データ分析:**  材料データから、材料の特性や性能を分析
* **AI・機械学習:**  材料の設計・開発を支援するAIや機械学習モデルの開発
* **シミュレーション:**  材料の性能を予測するためのシミュレーション

**MIのメリット:**

* **材料開発の効率化:**  従来の材料開発プロセスを大幅に短縮
* **新しい材料の発見:**  膨大なデータから、新しい材料の発見が可能
* **材料性能の向上:**  材料の性能をより正確に予測・設計
* **コスト削減:**  材料開発の費用を削減

## 実行環境のライブラリverを保存

In [35]:
!pip freeze > requirements_llm_rag.txt

In [36]:
from google.colab import files

files.download('requirements_llm_rag.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>